# Creating a parser function to manipulate gpx files

Our objecive is to create a function that can be fed *gpx* files and will compose a dataframe containing the most useful information of said route, as well as changing the name of the *gpx* file to one more easily accessed.

In [1]:
#Importing our libraries.

import pandas as pd
import time
import re
import pathlib
import os
import gpxpy
import gpxpy.gpx
import time
from pathlib import Path
import os
import random, string

In [ ]:
#Importing our list of names.

names = pd.read_csv('names.csv')
alpha = names['names'].tolist()

In [33]:
#Our parser function.

def parser(file, name):
    try:
        gpx_file = open(file, 'r') 
        gpx = gpxpy.parse(gpx_file) 
        coords = []
        for track in gpx.tracks:
            for segment in track.segments:        
                for point in segment.points:
                    coords.append(tuple([point.latitude, point.longitude]))
        parsed_file = {'name': name,
                       'original_name': track.name,
                       's_la': coords[0][0],
                       's_lo': coords[0][1],
                       'f_la': coords[-1][0],
                       'f_lo': coords[-1][1],
                        'source': gpx.creator,
                        'distance': track.length_3d()/1000, 
                        'climb': int(gpx.get_uphill_downhill()[0]),
                        'min_alt': int(gpx.get_elevation_extremes()[0]),
                        'max_alt': int(gpx.get_elevation_extremes()[1])}
        with open(name + ".gpx", "w") as f:
            f.write(gpx.to_xml())
        return parsed_file
    except:
        gpx_file = open(file, 'r', encoding='utf-8') 
        gpx = gpxpy.parse(gpx_file) 
        coords = []
        for track in gpx.tracks:
            for segment in track.segments:        
                for point in segment.points:
                    coords.append(tuple([point.latitude, point.longitude]))
        parsed_file = {'name': name,
                       'original_name': track.name,
                       's_la': coords[0][0],
                       's_lo': coords[0][1],
                       'f_la': coords[-1][0],
                       'f_lo': coords[-1][1],
                        'source': gpx.creator,
                        'distance': track.length_3d()/1000, 
                        'climb': int(gpx.get_uphill_downhill()[0]),
                        'min_alt': int(gpx.get_elevation_extremes()[0]),
                        'max_alt': int(gpx.get_elevation_extremes()[1])}
        with open(name + ".gpx", "w") as f:
            f.write(gpx.to_xml())
        return parsed_file

In [7]:
#Locating start and end of the track.

data = gpx.tracks[0].segments[0].points

start = [segment.points[0].latitude,segment.points[0].longitude]
end = [segment.points[-1].latitude,segment.points[-1].longitude]

NameError: name 'gpx' is not defined

In [35]:
#Testing it.

a = pd.DataFrame(parser('testfile.gpx', 'test'))
a.head()

ValueError: If using all scalar values, you must pass an index

## Name generation

In [51]:
#Generating a list of names using alphanumeric characters.

alpha = []

for i in range(50000000):
    x = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
    alpha.append(x)

In [121]:
#Dropping duplicate names.

alpha = set(alpha)

In [122]:
len(alpha)

49430049

In [61]:
#Storing our names as a dataframe for future use.

names = pd.DataFrame(set(alpha), columns =['names'])

In [63]:
#Storing it.

names.to_csv('names.csv', index=False)

In [150]:
def gpx_creator():
    """
    Input: none, but all target gpx files must be in a folder named 'gpx'.
    
    Output: new gpx files created as per 'parser' function.
    
    """
    start = time.time() #Starting our fimer.

    directory = 'gpx' #The folder containing the gpx files.
    parsed_list = []
    
    files = Path(directory).glob('*') #Using all files in the folder as input.
    i = 0
    for file in files:
        name = alpha[i]
        parsed_list.append(parser(file, name)) #Applying the previous function to every file.
        i = i+1

    stop = time.time() #Stopping our timer.
    duration = (stop - start) / 60
    
    df = pd.DataFrame(parsed_list)
    return df #Returning the elapsed minutes.

In [2]:
names = pd.read_csv('names.csv')
alpha = names['names'].tolist()

In [3]:
alpha = names['names'].tolist()